<a href="https://colab.research.google.com/github/racoope70/quant-trading-model-zoo/blob/main/PPO_Quantconnect_Converted_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from AlgorithmImports import (
    QCAlgorithm, Resolution, BrokerageName, AccountType,
    OrderEvent, OrderStatus, ConstantSlippageModel, PortfolioTarget
)
from collections import deque
import json, math
from datetime import datetime, timezone, timedelta
class ExternalSignalConsumer(QCAlgorithm):
    """
    Consumes model signals from a JSON endpoint and maps them to long-only weights.
    Improvements vs prior:
      • Sizing modes to avoid flat ~0.5 leverage (linear or thresholded).
      • Rolling 63d beta vs SPY.
      • Daily Sharpe/PSR plots & end-of-run summary (orders, fills, win rate).
    """
    def Initialize(self) -> None:
        #Backtest window
        self.SetStartDate(2024, 1, 1)
        self.SetEndDate(2024, 12, 31)
        self.SetCash(100000)
        self.mode           = (self.GetParameter("Mode") or "json-live").strip().lower()
        self.json_url       = (self.GetParameter("SignalsUrl") or "").strip()
        self.symbols_param  = (self.GetParameter("Symbols") or "UNH,TSLA,TMO").strip()
        self.poll_minutes   = int(self.GetParameter("PollingMinutes") or 60)
        self.sizing_mode     = (self.GetParameter("SizingMode") or "threshold").strip().lower()
        self.w_cap           = float(self.GetParameter("WeightCap") or 0.6)
        self.conf_floor      = float(self.GetParameter("ConfidenceFloor") or 0.60)
        self.ThrowIfEmptyUrl("SignalsUrl")
        self.symbols = {}
        for tkr in [s.strip().upper() for s in self.symbols_param.split(",") if s.strip()]:
            self.symbols[tkr] = self.AddEquity(tkr, Resolution.HOUR).Symbol
        self.SetSecurityInitializer(lambda sec: sec.SetSlippageModel(ConstantSlippageModel(0.01)))
        self.SetBrokerageModel(BrokerageName.INTERACTIVE_BROKERS_BROKERAGE, AccountType.MARGIN)
        self.UniverseSettings.Leverage = 1.0
        self.Settings.FreePortfolioValuePercentage = 0.05
        self.Settings.MinimumOrderMarginPortfolioPercentage = 0.0
        self.Settings.RebalancePortfolioOnSecurityChanges = False
        self.Schedule.On(
            self.DateRules.EveryDay(),
            self.TimeRules.Every(timedelta(minutes=self.poll_minutes)),
            self.PollJsonAndTrade
        )
        #Metrics / state
        self.fill_count = 0
        self.last_equity = float(self.Portfolio.TotalPortfolioValue)
        self.daily_returns = []
        self.valid_until_epoch = None
        self.last_poll_minute = None
        self.last_logged_sig = {}
        self.SetWarmup(5, Resolution.HOUR)
        self.spy = self.AddEquity("SPY", Resolution.DAILY).Symbol
        self.Schedule.On(
            self.DateRules.EveryDay(self.spy),
            self.TimeRules.AfterMarketClose(self.spy, 1),
            self.PushDailyMetrics
        )
        self.bench = self.spy
        self.SetBenchmark(self.bench)
        self._p_rets = deque(maxlen=63)
        self._b_rets = deque(maxlen=63)
        self._last_bench_close = None
        self._last_eod_date = None
        self._order_ids = set()
        self._fills = 0
        self.Log(f"Params OK | mode={self.mode} poll={self.poll_minutes} sizing={self.sizing_mode} "
                 f"cap={self.w_cap:.2f} floor={self.conf_floor:.2f} url={self.json_url[:60]}...")
    #Poll + Trade
    def PollJsonAndTrade(self) -> None:
        if self.IsWarmingUp:
            return
        cur_minute = int(self.Time.timestamp()) // 60
        if self.last_poll_minute == cur_minute:
            return
        self.last_poll_minute = cur_minute

        raw = self.Download(self.json_url)
        if not raw:
            if cur_minute % 10 == 0:
                self.Log("[WARN] Empty JSON from endpoint")
            return

        try:
            data = json.loads(raw)
        except Exception as e:
            if cur_minute % 10 == 0:
                self.Log(f"[WARN] JSON parse error: {e}")
            return

        vu_aware = self.ParseUtcAny(data.get("valid_until_utc") or data.get("valid_until"))
        self.valid_until_epoch = self.ToUtcEpoch(vu_aware) if vu_aware else None
        if self.mode == "json-live" and self.valid_until_epoch:
            if self.ToUtcEpoch(self.UtcTime) > self.valid_until_epoch:
                if cur_minute % 10 == 0:
                    self.Log(f"[STALE] now>{data.get('valid_until_utc')}, skipping")
                return
        models = data.get("models") or []
        by_sym = {(m.get("symbol") or "").upper(): m for m in models}
        weights = {}  #Symbol -> float weight in [0..w_cap]
        cnt_buy = cnt_sell = cnt_hold = 0
        for tkr, sym in self.symbols.items():
            m = by_sym.get(tkr)
            if not m:
                cnt_hold += 1
                continue
            sec = self.Securities[sym]
            if not sec.HasData or sec.Price <= 0 or not sec.Exchange.ExchangeOpen:
                cnt_hold += 1
                continue
            signal = (m.get("signal") or "").upper()
            conf = m.get("confidence")
            if conf is None:
                act = m.get("action")
                conf = abs(float(act)) if act is not None else float(m.get("p_long") or 0.0)
            conf = float(conf or 0.0)
            conf = max(0.0, min(1.0, conf))
            if signal == "BUY":
                if self.sizing_mode == "linear":
                    w = self.w_cap * conf
                else:
                    if conf >= self.conf_floor:
                        w = self.w_cap * (conf - self.conf_floor) / (1.0 - self.conf_floor)
                    else:
                        w = 0.0
                w = max(0.0, min(self.w_cap, w))
                weights[sym] = w
                cnt_buy += 1
            elif signal == "SELL":
                weights[sym] = 0.0
                cnt_sell += 1
            else:
                weights[sym] = 0.0
                cnt_hold += 1
            if self.last_logged_sig.get(tkr) != signal:
                self.last_logged_sig[tkr] = signal
                self.Log(f"{tkr}: {signal} (conf={conf:.2f})")
        if not weights:
            return
        cap = 1.0 - self.Settings.FreePortfolioValuePercentage
        gross = sum(max(0.0, w) for w in weights.values())
        if gross > cap and gross > 0:
            scale = cap / gross
            for s in list(weights):
                weights[s] *= scale
        targets = [PortfolioTarget(sym, w) for sym, w in weights.items()]
        self.SetHoldings(targets)
        self.Log(f"Rebalance: buys={cnt_buy}, sells={cnt_sell}, holds={cnt_hold}, gross={sum(abs(w) for w in weights.values()):.2f}")
    def OnOrderEvent(self, orderEvent: OrderEvent) -> None:
        #track orders/fills for the report
        self._order_ids.add(orderEvent.OrderId)
        if orderEvent.Status == OrderStatus.Filled:
            self._fills += 1
        if orderEvent.Status != OrderStatus.Filled:
            return
        self.fill_count += 1
        self.Plot("Execs", "Fills", self.fill_count)
        closed = list(self.TradeBuilder.ClosedTrades)
        wins   = sum(1 for t in closed if t.ProfitLoss >  0)
        losses = sum(1 for t in closed if t.ProfitLoss <= 0)
        self.Plot("Execs", "ClosedWins",  wins)
        self.Plot("Execs", "ClosedLosses", losses)
    def OnEndOfDay(self) -> None:
        d = self.Time.date()
        if self._last_eod_date == d:
            return
        self._last_eod_date = d
        cur = float(self.Portfolio.TotalPortfolioValue)
        r = 0.0
        if self.last_equity > 0:
            r = (cur / self.last_equity) - 1.0
            self.daily_returns.append(r)
            if len(self.daily_returns) > 400:
                self.daily_returns = self.daily_returns[-300:]
        b_close = float(self.Securities[self.bench].Close)
        b_ret = 0.0 if not self._last_bench_close else (b_close / self._last_bench_close - 1.0)
        self._last_bench_close = b_close
        self._p_rets.append(r)
        self._b_rets.append(b_ret)
        if len(self._b_rets) >= 20:
            mpr = sum(self._p_rets) / len(self._p_rets)
            mbr = sum(self._b_rets) / len(self._b_rets)
            cov = sum((pr - mpr) * (br - mbr) for pr, br in zip(self._p_rets, self._b_rets)) / max(len(self._b_rets) - 1, 1)
            var = sum((br - mbr) ** 2 for br in self._b_rets) / max(len(self._b_rets) - 1, 1)
            beta = cov / var if var > 0 else 0.0
            self.Plot("Risk", "Beta(63d)", beta)
        self.last_equity = cur
    def PushDailyMetrics(self) -> None:
        n = len(self.daily_returns)
        if n < 10:
            return

        mean = sum(self.daily_returns) / n
        var = sum((x - mean) ** 2 for x in self.daily_returns) / max(n - 1, 1)
        sd = math.sqrt(max(var, 1e-12))

        sharpe = (mean / sd) * math.sqrt(252.0) if sd > 0 else 0.0
        z = (mean / sd) * math.sqrt(n) if sd > 0 else 0.0
        psr = 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

        self.Plot("Risk", "Sharpe(rolling)", sharpe)
        self.Plot("Risk", "PSR_vs0", psr)

        closed = list(self.TradeBuilder.ClosedTrades)
        if closed:
            wins = sum(1 for t in closed if t.ProfitLoss > 0)
            winrate = wins / float(len(closed))
            self.Plot("Risk", "WinRate", winrate)
    def OnEndOfAlgorithm(self) -> None:
        closed = list(self.TradeBuilder.ClosedTrades)
        wins = sum(1 for t in closed if t.ProfitLoss > 0)
        win_rate = (wins / len(closed)) if closed else float('nan')
        self.Log(
            f"RUN_SUMMARY | orders={len(self._order_ids)} | "
            f"fills={self._fills} | closed_trades={len(closed)} | "
            f"win_rate={win_rate:.2%}"
        )
    def ThrowIfEmptyUrl(self, which: str) -> None:
        if not self.json_url:
            raise ValueError(f"Parameter '{which}' not provided. Set it in Project → Parameters.")
    def ParseUtcAny(self, s):
        if not s:
            return None
        s = str(s)
        try:
            s2 = s.replace("Z", "+00:00")
            dt = datetime.fromisoformat(s2)
            if dt.tzinfo is None:
                return dt.replace(tzinfo=timezone.utc)
            return dt.astimezone(timezone.utc)
        except Exception:
            pass
        try:
            return datetime.fromtimestamp(float(s), tz=timezone.utc)
        except Exception:
            return None
    def ToUtcEpoch(self, dt):
        if dt is None:
            return None
        if isinstance(dt, datetime):
            if dt.tzinfo is None:
                dt = dt.replace(tzinfo=timezone.utc)
            return dt.timestamp()
        try:
            return float(dt)
        except Exception:
            return None